In [1]:
import findspark

In [2]:
import pyspark

In [3]:
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [7]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
sc=SparkSession.builder.appName('project').master('local').getOrCreate()
sc

In [8]:
schema = StructType([
    StructField('Movie_name',StringType(),True),
    StructField('Language',StringType(),True),
    StructField('Imdb_rating',FloatType(),True),
    StructField('Running_time',StringType(),True),
    StructField('Year_of_release',IntegerType(),True),
    StructField('Maturity_rating',StringType(),True),
    StructField('Plot',StringType(),True),
   
])

In [15]:
amazon_df=sc.read.csv('amazon prime movies.csv',schema=schema)

In [17]:
amazon_df.show()

+--------------------+--------+-----------+------------+---------------+---------------+--------------------+
|          Movie_name|Language|Imdb_rating|Running_time|Year_of_release|Maturity_rating|                Plot|
+--------------------+--------+-----------+------------+---------------+---------------+--------------------+
|          John Rambo|   Tamil|        7.0|  1 h 26 min|           2008|            18+|In Thailand, John...|
|        American Pie| English|        7.0|  1 h 35 min|           1999|            18+|"You'll never loo...|
|           Bombshell| English|        6.8|  1 h 48 min|           2019|            18+|The provocative r...|
|          Love Birds|   Tamil|        5.1|  2 h 40 min|           1996|            All|After accidentall...|
|               Hippi|  Telugu|        5.0|  2 h 20 min|           2019|            18+|Hippi is about th...|
|Honey Bunny As Su...|   Tamil|       null|       49min|           2018|             7+|Flamingo a Hitech...|
|         

In [18]:
amazon_df.count()

8129

In [22]:
amazon_df=amazon_df.drop_duplicates()

In [23]:
amazon_df.count()

8126

In [25]:
amazon_df=amazon_df.drop('all')

In [26]:
amazon_df.count()

8126

## 1.  Review the movie names and languages names in the year of 2019.

In [27]:
result1=amazon_df.filter(col('Year_of_release')==2019).select('Movie_name','Language')

In [28]:
result1.show(truncate=False)

+-------------------------------------+---------+
|Movie_name                           |Language |
+-------------------------------------+---------+
|Unmadiyude Maranam | ഉന്മാദിയുടെ മരണം|English  |
|Majili                               |Telugu   |
|Carl's Car Wash 2                    |English  |
|Kathanam                             |Tamil    |
|The Bumble Nums - Season 3           |English  |
|Bakrid                               |Tamil    |
|Saaho (Telugu)                       |Telugu   |
|Mighty Raju - Chutti Ho Gayi         |Hindi    |
|Kettiyolaanu Ente Malakha            |Malayalam|
|Captive State                        |English  |
|Bharaate                             |Kannada  |
|Gone Kesh                            |Hindi    |
|Family 420 Once Again                |Punjabi  |
|Viswasam                             |Tamil    |
|The Bumble Nums 2 - Super Simple     |English  |
|Nanna Prakara                        |Kannada  |
|Motherless Brooklyn                  |English  |


## 2.Which are the languages, movies highly watched by people of all ages? And in this scenrio, give the movie names of that language


In [29]:
all_filtered=amazon_df.filter(col('Maturity_rating')=='All')

In [31]:
all_filtered.show()

+--------------------+---------+-----------+------------+---------------+---------------+--------------------+
|          Movie_name| Language|Imdb_rating|Running_time|Year_of_release|Maturity_rating|                Plot|
+--------------------+---------+-----------+------------+---------------+---------------+--------------------+
|           Dahavi Fa|  Marathi|        7.8| 2 h 23 mins|           2002|            All|Fed up with gradi...|
|    Top Cat (Telugu)|   Telugu|        4.6|  1 h 26 min|           2012|            All|Top Cat Is An Awa...|
|Dharmam Thalai Ka...|    Tamil|        7.3|   2 h 5 min|           1963|            All|Masked killers su...|
| Kshanbhar Vishranti|  Marathi|        6.3| 2 h 23 mins|           2010|            All|It is the story o...|
|          Ulladakkam|Malayalam|        7.2|          2h|           1991|            All|Doctor Sunny, a p...|
|       Dil Ka Rishta|    Hindi|        4.7|  2 h 23 min|           2003|            All|Jai unintentional...|
|

In [36]:
max_rating=all_filtered.groupBy(col('Language')).agg(count('Maturity_rating').alias('count')).orderBy(col('count').desc())

In [38]:
max_rating.show()

+---------+-----+
| Language|count|
+---------+-----+
|    Hindi|  615|
|    Tamil|  225|
|Malayalam|  198|
|  Marathi|  193|
|   Telugu|  186|
|  Bengali|  125|
|  English|   94|
| Gujarati|   57|
|  Punjabi|   44|
|  Kannada|   24|
+---------+-----+



In [41]:
max_rating1=max_rating.head()[0]

In [43]:
all_filtered.filter(col('Language')==max_rating1,).orderBy(col('Imdb_rating').desc()).show()

+--------------------+--------+-----------+------------+---------------+---------------+--------------------+
|          Movie_name|Language|Imdb_rating|Running_time|Year_of_release|Maturity_rating|                Plot|
+--------------------+--------+-----------+------------+---------------+---------------+--------------------+
|             Pushpak|   Hindi|        8.7|  1 h 54 min|           1987|            All|An unemployed you...|
|            Gol Maal|   Hindi|        8.6|  2 h 17 min|           1979|            All|Ramprasad works f...|
|             Baarish|   Hindi|        8.6|  1 h 50 min|           null|            All|Baarish Is A Crim...|
|              Pyaasa|   Hindi|        8.5|  2 h 23 min|           null|            All|Struggling Poet V...|
|             Samadhi|   Hindi|        8.5|  2 h 18 min|           1972|            All|Lakhan Singh give...|
|  Jaane Bhi Do Yaaro|   Hindi|        8.4|  2 h 11 min|           1983|            All|Two photographers...|
|Petrichor

## 3.Which age group (maturity rating) is watching hindi movies? also, the movie titles.

In [44]:
age_group=amazon_df.filter(col('Language')=='Hindi').groupBy(col('Maturity_rating')).agg(count('Maturity_rating').alias('count')).orderBy(col('count').desc())

In [51]:
age_group.show()

+---------------+-----+
|Maturity_rating|count|
+---------------+-----+
|            All|  615|
|             NR|  499|
|            13+|  491|
|            18+|  258|
|            16+|   62|
|             7+|   57|
+---------------+-----+



In [52]:
all_group_hindi=age_group.head()[0]

In [53]:
all_group_hindi

'All'

In [85]:
result3=amazon_df.filter(col('Language')=='Hindi').filter(col('Maturity_rating')==all_group_hindi).orderBy(col('Imdb_rating').desc())

In [86]:
result3.show()

+--------------------+--------+-----------+------------+---------------+---------------+--------------------+
|          Movie_name|Language|Imdb_rating|Running_time|Year_of_release|Maturity_rating|                Plot|
+--------------------+--------+-----------+------------+---------------+---------------+--------------------+
|             Pushpak|   Hindi|        8.7|  1 h 54 min|           1987|            All|An unemployed you...|
|            Gol Maal|   Hindi|        8.6|  2 h 17 min|           1979|            All|Ramprasad works f...|
|             Baarish|   Hindi|        8.6|  1 h 50 min|           null|            All|Baarish Is A Crim...|
|              Pyaasa|   Hindi|        8.5|  2 h 23 min|           null|            All|Struggling Poet V...|
|             Samadhi|   Hindi|        8.5|  2 h 18 min|           1972|            All|Lakhan Singh give...|
|  Jaane Bhi Do Yaaro|   Hindi|        8.4|  2 h 11 min|           1983|            All|Two photographers...|
|Petrichor

## 4.Which movie has the highest running duration and in which year? 

In [71]:
timehr_df=amazon_df.withColumn('hour', when(regexp_extract(col('Running_time'),r'(\d+) h',1)!='',\
                                            regexp_extract(col('Running_time'),r'(\d+) h', 1)).\
                               when(regexp_extract(col('Running_time'),r'(\d+)h',1)!='',\
                                    regexp_extract(col('Running_time'),r'(\d+)h',1)).otherwise(0).cast('int'))\
    .withColumn('min', when(regexp_extract(col('Running_time'),r'(\d+) min',1)!='',\
                                            regexp_extract(col('Running_time'),r'(\d+) min', 1)).\
                               when(regexp_extract(col('Running_time'),r'(\d+)min',1)!='',\
                                    regexp_extract(col('Running_time'),r'(\d+)min',1)).otherwise(0).cast('int'))

In [73]:
timehr_min=timehr_df.withColumn('total_time',col('hour')*60+col('min'))

In [74]:
timehr_min.show()

+--------------------+---------+-----------+------------+---------------+---------------+--------------------+----+---+----------+
|          Movie_name| Language|Imdb_rating|Running_time|Year_of_release|Maturity_rating|                Plot|hour|min|total_time|
+--------------------+---------+-----------+------------+---------------+---------------+--------------------+----+---+----------+
|          Noor Jahan|  Bengali|        5.3|  2 h 22 min|           2018|            13+|A Bengali romanti...|   2| 22|       142|
|          Rathaavara|  Kannada|        6.5|  2 h 11 min|           2015|            13+|Manikantha (RaviS...|   2| 11|       131|
|       Suicide Squad|  English|        6.0|   2 h 2 min|           2016|            13+|A secret governme...|   2|  2|       122|
|           Dahavi Fa|  Marathi|        7.8| 2 h 23 mins|           2002|            All|Fed up with gradi...|   2| 23|       143|
|         Air Hostess|Malayalam|        5.2|  2 h 21 min|           1980|          

In [76]:
max_duration=timehr_min.select(max('total_time')).head()[0]

In [77]:
max_duration

255

In [83]:
result4=timehr_min.filter(col('total_time')==max_duration)

In [84]:
result4.show()

+--------------------+--------+-----------+------------+---------------+---------------+--------------------+----+---+----------+
|          Movie_name|Language|Imdb_rating|Running_time|Year_of_release|Maturity_rating|                Plot|hour|min|total_time|
+--------------------+--------+-----------+------------+---------------+---------------+--------------------+----+---+----------+
|’Rameau’s Nephew’...| English|       null|  4 h 15 min|           1974|             NR|Consisting of 25 ...|   4| 15|       255|
+--------------------+--------+-----------+------------+---------------+---------------+--------------------+----+---+----------+



## 5. And review the total number of movie ratings by language wise, and what are the movies that are high maturity rating?

In [81]:
result5=amazon_df.groupBy('Language','Imdb_rating').agg(count('Imdb_rating').alias('ratings_count')).orderBy(col('ratings_count').desc())

In [82]:
result5.show()

+--------+-----------+-------------+
|Language|Imdb_rating|ratings_count|
+--------+-----------+-------------+
| English|        6.5|           65|
|   Hindi|        6.5|           60|
| English|        6.2|           59|
|   Hindi|        7.2|           57|
|   Hindi|        6.2|           57|
|   Hindi|        7.1|           56|
|   Hindi|        7.0|           56|
|   Hindi|        6.7|           55|
|   Hindi|        6.9|           54|
|   Hindi|        6.4|           54|
|   Hindi|        6.6|           54|
| English|        7.0|           53|
|   Hindi|        6.3|           53|
|   Hindi|        6.0|           52|
|   Hindi|        6.8|           50|
| English|        6.4|           50|
| English|        6.6|           49|
| English|        7.3|           46|
| English|        6.1|           46|
| English|        6.3|           46|
+--------+-----------+-------------+
only showing top 20 rows

